In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [3]:
key_read  = '4207277A-2EBD-11ED-B5AA-42010A800006'

In [4]:
sleep_seconds = 3

In [5]:
def get_sensorslist(nwlng,nwlat,selng,selat,location,key_read):
    # PurpleAir API URL
    root_url = 'https://api.purpleair.com/v1/sensors/'

    # Box domain: lat_lon = [nwlng,, nwlat, selng, selat]
    lat_lon = [nwlng, nwlat, selng, selat]
    for i,l in enumerate(lat_lon):
        if (i == 0):
            ll_api_url = f'&nwlng={l}'
        elif (i == 1):
            ll_api_url += f'&nwlat={l}'
        elif (i == 2):
            ll_api_url += f'&selng={l}'
        elif (i == 3):
            ll_api_url += f'&selat={l}'
        
    # Fields to get
    fields_list = ["sensor_index","date_created", "private","name","icon","location_type","model","hardware","latitude","longitude"] 
    for i,f in enumerate(fields_list):
        if (i == 0):
            fields_api_url = f'&fields={f}'
        else:
            fields_api_url += f'%2C{f}'

    # Indoor, outdoor or all
    if (location == 'indoor'):
        loc_api = f'&location_type=1'
    elif (location == 'outdoor'):
        loc_api = f'&location_type=0'
    else:
        loc_api = ''
            
    # Final API URL
    api_url = root_url + f'?api_key={key_read}' + fields_api_url + ll_api_url + loc_api

    # Getting data
    response = requests.get(api_url)

    if response.status_code == 200:
        #print(response.text)
        json_data = json.loads(response.content)['data']
        df = pd.DataFrame.from_records(json_data)
        df.columns = fields_list
    else:
        raise requests.exceptions.RequestException

    # writing to csv file
    filename = 'sensors_list.csv'
    df.to_csv(filename, index=False, header=True)
            
    # Creating a Sensors 
    sensorslist = list(df.sensor_index)
    
    return sensorslist

In [6]:
# A rough boundary of CA
nwlng = -125.046141
nwlat = 42.203915
selng = -114.015748
selat = 32.814535
location = 'outdoor'

In [7]:
sensorList = get_sensorslist(nwlng,nwlat,selng,selat,location,key_read)

In [8]:
len(sensorList)

8239

In [9]:
print(sensorList[:10])

[131075, 131079, 131077, 131083, 131087, 131085, 131091, 131089, 131097, 131203]


In [10]:
def get_historicaldata(sensors_list,bdate,edate,average_time,key_read):
    # Create an empty dataframe
    df1 = pd.DataFrame()
    
    # Historical API URL
    root_api_url = 'https://api.purpleair.com/v1/sensors/'
    
    # Average time: The desired average in minutes, one of the following:0 (real-time),10 (default if not specified),30,60
    average_api = f'&average={average_time}'

    # Creating fields api url from fields list to download the data: Note: Sensor ID/Index will not be downloaded as default
    fields_list = ['pm2.5_atm_a', 'pm2.5_atm_b', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b', 'humidity_a', 'humidity_b', 
               'temperature_a', 'temperature_b', 'pressure_a', 'pressure_b', 'pm1.0_atm', 'pm1.0_atm_a', 'pm1.0_atm_b', 
                   'pm1.0_cf_1', 'pm1.0_cf_1_a', 'pm1.0_cf_1_b', 'pm2.5_alt', 'pm2.5_alt_a', 'pm2.5_alt_b', 
                   'pm2.5_atm', 'pm2.5_atm_a', 'pm2.5_atm_b', 'pm2.5_cf_1', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b', 
                   'pm10.0_atm', 'pm10.0_atm_a', 'pm10.0_atm_b', 'pm10.0_cf_1', 'pm10.0_cf_1_a', 
                   'pm10.0_cf_1_b', 'scattering_coefficient', 'scattering_coefficient_a', 'scattering_coefficient_b',
                   'deciviews, deciviews_a', 'deciviews_b', 'visual_range', 'visual_range_a', 
                   'visual_range_b', '0.3_um_count', '0.3_um_count_a', '0.3_um_count_b', 
                   '0.5_um_count', '0.5_um_count_a', '0.5_um_count_b', '1.0_um_count', '1.0_um_count_a',
                   '1.0_um_count_b', '2.5_um_count', '2.5_um_count_a', '2.5_um_count_b', '5.0_um_count', 
                   '5.0_um_count_a', '5.0_um_count_b', '10.0_um_count', '10.0_um_count_a', '10.0_um_count_b']
    for i,f in enumerate(fields_list):
        if (i == 0):
            fields_api_url = f'&fields={f}'
        else:
            fields_api_url += f'%2C{f}'

    # Dates of Historical Data period
    begindate = datetime.strptime(bdate, '%m-%d-%Y')
    enddate   = datetime.strptime(edate, '%m-%d-%Y')
    
    # Downlaod days based on average
    if (average_time == 60):
        date_list = pd.date_range(begindate,enddate,freq='14d') # for 14 days of data
    else:
        date_list = pd.date_range(begindate,enddate,freq='2d') # for 2 days of data
        
    # Converting to UNIX timestamp
    date_list_unix=[]
    for dt in date_list:
        date_list_unix.append(int(time.mktime(dt.timetuple())))

    # Reversing to get data from end date to start date
    date_list_unix.reverse()
    len_datelist = len(date_list_unix) - 1
        
    # Getting 2-data for one sensor at a time
    for s in sensors_list:
        # Adding sensor_index & API Key
        hist_api_url = root_api_url + f'{s}/history/csv?api_key={key_read}'

        # Creating start and end date api url
        for i,d in enumerate(date_list_unix):
            # Wait time 
            time.sleep(sleep_seconds)
            
            if (i < len_datelist):
                print('Downloading for PA: %s for Dates: %s and %s.' 
                      %(s,datetime.fromtimestamp(date_list_unix[i+1]),datetime.fromtimestamp(d)))
                dates_api_url = f'&start_timestamp={date_list_unix[i+1]}&end_timestamp={d}'
            
                # Final API URL
                api_url = hist_api_url + dates_api_url + average_api + fields_api_url
                            
                #
                try:
                    response = requests.get(api_url)
                except:
                    print(api_url)
                #
                try:
                    assert response.status_code == requests.codes.ok
                
                    # Creating a Pandas DataFrame
                    df = pd.read_csv(StringIO(response.text), sep=",", header=0)
                
                except AssertionError:
                    df = pd.DataFrame()
                    print('Bad URL!')
            
                if df.empty:
                    print('------------- No Data Available -------------')
                else:
                    # Adding Sensor Index/ID
                    df['id'] = s
                
                    #
                    date_time_utc=[]
                    for index, row in df.iterrows():
                        date_time_utc.append(datetime.fromtimestamp(row['time_stamp']))
                    df['date_time_utc'] = date_time_utc
                
                    # Dropping duplicate rows
                    df = df.drop_duplicates(subset=None, keep='first', inplace=False)
                    df2 = pd.concat([df1, df], axis = 0)
                    df1 = df2
                    
    df1.to_csv('Yearly_Historical_Data.csv')

## Find the sensors that were created before 01-01-2016

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [12]:
ur_files = ddf.read_csv('sensors_list.csv')
sensors = ur_files.compute()
sensors.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
0,131075,1632955574,0,Mariners Bluff,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,33.515110,-117.67972
1,131079,1632955612,0,BRSKBV-outside,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,37.753150,-122.44364
2,131077,1632955594,0,BEE Patio,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,37.932730,-122.03972
3,131083,1632955622,0,Bettencourt Home,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,38.287594,-122.46281
4,131087,1632955644,0,Graeagle Meadows,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,39.762830,-120.61008


In [13]:
sensors['date_created'].dtype

dtype('int64')

In [14]:
date_list = list(sensors['date_created'])
date_list[:10]

[1632955574,
 1632955612,
 1632955594,
 1632955622,
 1632955644,
 1632955638,
 1632955657,
 1632955649,
 1633028780,
 1633034622]

In [15]:
valid_sensors = []
for element in date_list:
    if element < 1451635201:
        valid_sensors.append(element)

In [16]:
len(valid_sensors)

0

In [17]:
sensors_2016 = sensors[sensors['date_created'] < 1451635201]
sensors_2016.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude


In [18]:
sensors_2017 = sensors[sensors['date_created'] < 1483257601]
sensors_2017.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
11,195,1470083084,0,Trinidad,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,41.059727,-124.128890
14,407,1476477823,0,CCA Motor Parts of America,0,0,PA-II,2.0+BME280+PMSX003A+PMSX003B,33.842000,-118.337690
15,417,1476482843,0,CCA Hamilton and Silvius,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.716675,-118.309906
16,473,1478548774,0,Sunnyside 93727,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,36.724483,-119.701645
17,489,1480104897,0,CCA Lakme an Q,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.795550,-118.260940


In [19]:
len(sensors_2017)

9

In [20]:
sensors_2018 = sensors[sensors['date_created'] < 1514793601]
sensors_2018.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
11,195,1470083084,0,Trinidad,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,41.059727,-124.128890
14,407,1476477823,0,CCA Motor Parts of America,0,0,PA-II,2.0+BME280+PMSX003A+PMSX003B,33.842000,-118.337690
15,417,1476482843,0,CCA Hamilton and Silvius,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.716675,-118.309906
16,473,1478548774,0,Sunnyside 93727,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,36.724483,-119.701645
17,489,1480104897,0,CCA Lakme an Q,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.795550,-118.260940


In [21]:
len(sensors_2018)

205

In [22]:
sensors_2019 = sensors[sensors['date_created'] < 1546329601]
sensors_2019.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
11,195,1470083084,0,Trinidad,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,41.059727,-124.128890
14,407,1476477823,0,CCA Motor Parts of America,0,0,PA-II,2.0+BME280+PMSX003A+PMSX003B,33.842000,-118.337690
15,417,1476482843,0,CCA Hamilton and Silvius,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.716675,-118.309906
16,473,1478548774,0,Sunnyside 93727,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,36.724483,-119.701645
17,489,1480104897,0,CCA Lakme an Q,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.795550,-118.260940


In [23]:
len(sensors_2019)

1175

In [24]:
sensors_2020 = sensors[sensors['date_created'] < 1577865601]
sensors_2020.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
11,195,1470083084,0,Trinidad,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,41.059727,-124.128890
14,407,1476477823,0,CCA Motor Parts of America,0,0,PA-II,2.0+BME280+PMSX003A+PMSX003B,33.842000,-118.337690
15,417,1476482843,0,CCA Hamilton and Silvius,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.716675,-118.309906
16,473,1478548774,0,Sunnyside 93727,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,36.724483,-119.701645
17,489,1480104897,0,CCA Lakme an Q,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.795550,-118.260940


In [25]:
len(sensors_2020)

1747

In [26]:
sensors_2021 = sensors[sensors['date_created'] < 1609488001]
sensors_2021.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
11,195,1470083084,0,Trinidad,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,41.059727,-124.128890
14,407,1476477823,0,CCA Motor Parts of America,0,0,PA-II,2.0+BME280+PMSX003A+PMSX003B,33.842000,-118.337690
15,417,1476482843,0,CCA Hamilton and Silvius,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.716675,-118.309906
16,473,1478548774,0,Sunnyside 93727,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,36.724483,-119.701645
17,489,1480104897,0,CCA Lakme an Q,0,0,PA-II,2.0+1M+BME280+PMSX003-B+PMSX003-A,33.795550,-118.260940


In [27]:
len(sensors_2021)

5738

In [28]:
sensors_2022 = sensors[sensors['date_created'] < 1641024001]
sensors_2022.head()

,sensor_index,date_created,private,name,icon,location_type,model,hardware,latitude,longitude
0,131075,1632955574,0,Mariners Bluff,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,33.515110,-117.67972
1,131079,1632955612,0,BRSKBV-outside,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,37.753150,-122.44364
2,131077,1632955594,0,BEE Patio,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,37.932730,-122.03972
3,131083,1632955622,0,Bettencourt Home,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,38.287594,-122.46281
4,131087,1632955644,0,Graeagle Meadows,0,0,PA-II,2.0+BME280+PMSX003-B+PMSX003-A,39.762830,-120.61008


In [29]:
len(sensors_2022)

7848

### Find the data in 2021

In [30]:
# Data download period
bdate = '1-1-2017' 
edate = '12-31-2017'

In [31]:
sensorList_2017 = list(sensors_2017['sensor_index'])

In [32]:
average_time=1440

In [33]:
get_historicaldata(sensorList_2017,bdate,edate,average_time,key_read)

------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available -------------
------------- No Data Available --